In [ ]:
import os
import sys
import csv

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
def set_font_small():
    plt.style.use('default') 
    sns.set_style('darkgrid')
    mpl.rcParams.update({'font.size': 20})  
def set_font_large():
    plt.style.use('default') 
    sns.set_style('darkgrid')
    mpl.rcParams.update({'font.size': 30}) 
    plt.rc('font', size=30)
    plt.rc('axes', labelsize=30)    
    plt.rc('xtick', labelsize=30)    
    plt.rc('ytick', labelsize=30)    
    plt.rc('legend', fontsize=30)    
set_font_small()

In [ ]:
# Define how to get the timestamps
USE_TIMESTAMP = 0
USE_DT_MICRO = 1
USE_DEVICE_STAMP = 2
USE_ASSUME_TICK_US = 3

TIME_METRIC = USE_DEVICE_STAMP

In [ ]:
# Smoothening function with rolling log
def smoothen_line(x, y, smoothen_step):
    xlen = int((x[-1] - x[0] + 1) / smoothen_step) 
    newx = [float(i) * smoothen_step + x[0] for i in range(xlen)]
    newy = [[0,0] for i in range(xlen)]

    # Count into bins
    first = (x[0] // smoothen_step) * smoothen_step
    for i,j in zip(x,y):
        roundedi = (i // smoothen_step) * smoothen_step + smoothen_step * 0.5
        roundedi = int((roundedi - first) / smoothen_step)
        newy[roundedi] = [newy[roundedi][0] + 1, newy[roundedi][1] + j]
    
    # Collapse
    newy = [i[1] / i[0] if i[0] else 0 for i in newy]
    return (newx, newy)

# Get the max value in a window
def upperbound_line(x, y, smoothen_step):
    xlen = int((x[-1] - x[0] + 1) / smoothen_step) 
    newx = [float(i) * smoothen_step + x[0] for i in range(xlen)]
    newy = [0. for i in range(xlen)]

    # Count into bins
    first = (x[0] // smoothen_step) * smoothen_step
    for i,j in zip(x,y):
        roundedi = (i // smoothen_step) * smoothen_step + smoothen_step * 0.5
        roundedi = int((roundedi - first) / smoothen_step)
        newy[roundedi] = max(newy[roundedi], j)
    
    return (newx, newy)

# Get the minimum value in a window
def lowerbound_line(x, y, smoothen_step):
    xlen = int((x[-1] - x[0] + 1) / smoothen_step) 
    newx = [float(i) * smoothen_step + x[0] for i in range(xlen)]
    newy = [100. for i in range(xlen)]

    # Count into bins
    first = (x[0] // smoothen_step) * smoothen_step
    for i,j in zip(x,y):
        roundedi = (i // smoothen_step) * smoothen_step + smoothen_step * 0.5
        roundedi = int((roundedi - first) / smoothen_step)
        newy[roundedi] = min(newy[roundedi], j)
    
    # clamp
    i = len(newy) - 1
    while (i >= 0):
        newy[i] = 0. if newy[i] > 99. else newy[i]
        i = i -1

    return (newx, newy)

# Parse a ".txt" file from psrun with the chosen timefunction, export it as time:power, and filter on the specified timerange
def parse_file(filename, timerange, timefunction):
    # Parse
    data = np.genfromtxt(f'{filename}.txt', dtype=None, names=True, delimiter=' ', unpack=True, usecols=\
        (
            "time",
            "dt_micro",
            "device_timestamp",
            "power_total"
        ))
    # Filter
    if timefunction is USE_DEVICE_STAMP:
        dt_parsed = np.array([0.] * len(data[0]))
        since     = data[0][0] if len(data) else 0
        since_dev = data[2][0] if len(data) else 0
        for idx, j in enumerate(data[2]):
            if not idx:
                dt_parsed[idx] = since
                # print(idx, since, data[0][idx])
                continue
            jump = j - since_dev if j > since_dev else 1024 - since_dev + j 
            since = since + float(jump) / 1_000_000          
            dt_parsed[idx] = since
            # print(idx, since, data[0][idx], '---', j, since_dev, jump)
            since_dev = j
            # if idx > 10000:
            #     exit(1)
        # Filter
        timespan = np.logical_and(dt_parsed >= timerange[0], dt_parsed <= timerange[1]) 
        return (dt_parsed[timespan], data[3][timespan])
    elif timefunction is USE_ASSUME_TICK_US:
        dt_parsed = np.array([0.] * len(data[0]))
        since = data[0][0] if len(data) else 0
        for idx, j in enumerate(data[1]):
            dt_parsed[idx] = since
            since = since + 50 / 1_000_000          
        # Filter
        timespan = np.logical_and(dt_parsed >= timerange[0], dt_parsed <= timerange[1]) 
        return (dt_parsed[timespan], data[3][timespan])
    elif timefunction is USE_DT_MICRO:
        # Create new array
        dt_parsed = np.array([0.] * len(data[0]))
        since = 0
        for idx, j in enumerate(data[1]):
            dt_parsed[idx] = since
            since = since + float(j) / 1_000_000          
        # Filter
        timespan = np.logical_and(dt_parsed >= timerange[0], dt_parsed <= timerange[1]) 
        return (dt_parsed[timespan], data[3][timespan])
    else:
        # We can use the host as is
        timespan = np.logical_and(data[0] >= timerange[0], data[0] <= timerange[1]) 
        return (data[0][timespan], data[3][timespan])

In [ ]:
def plot_gc(size_label):
    # Plot 1
    IN_FILES = ['data/steadylower']
    BW_FILE  = 'data/steadylower_bw.1.log'
    OUT_FILE = f'ssd_power_measurement_gc_{size_label}'
    
    X_RANGE = (0, 1200)
    Y_RANGE = (0.00, 10.00)
    X_SMOOTHENING_STEP = 1
    
    fig, ax1 = plt.subplots(figsize=(15, 7))
    ax2 = ax1.twinx()
    num_lines = len(IN_FILES) * 3
    greys = iter(sns.color_palette("colorblind", num_lines))
    
    ln1 = None
    for filename in IN_FILES:
        x, y = parse_file(filename, X_RANGE, TIME_METRIC)
        xs, ys = smoothen_line(x, y, X_SMOOTHENING_STEP)
        xu, yu = upperbound_line(x, y, X_SMOOTHENING_STEP)
        xl, yl = lowerbound_line(x, y, X_SMOOTHENING_STEP)
        ln1 = sns.lineplot(x=xs, y=ys, c=next(greys), label='Power', ax=ax1)
    
    bw_x = []
    bw_y = []
    ln2 = None
    with open(BW_FILE, 'r') as file:
        data = [line.rstrip() for line in file]
        bw_x = [0.] * len(data)
        bw_y = [0.] * len(data)
        for idx, line in enumerate(data):
            spl = line.split()
            bw_x[idx] = (float(spl[0][:-1])) / 1000.
            bw_y[idx] = (float(spl[1][:-1])) / 1024.
        bw_x = np.array(bw_x)
        bw_y = np.array(bw_y)
        timespan = np.logical_and(bw_x >= X_RANGE[0], bw_x <= X_RANGE[1]) 
        bw_x = bw_x[timespan]
        bw_y = bw_y[timespan]
    
        xs, ys = smoothen_line(bw_x, bw_y, X_SMOOTHENING_STEP)
        ln2 = sns.lineplot(x=xs, y=ys, c=next(greys), label='Bandwidth', ax=ax2)
    
    ax1.set_ylim([0, 7])
    ax1.set_yticks((0, 1, 2, 3, 4, 5, 6, 7))
    # ax1.set_xticks()
    plt.xticks([0, 300, 600, 900, 1200], ['0', '5', '10', '15', '20'])
    ax2.set_ylim([0, 1024])
    ax2.set_yticks((0, 250, 500, 750, 1000))
    # ax2.set_yticklabels(['0', '1', '2', '3'])
    ax1.set_xlabel("Time (m)")
    ax1.set_ylabel("Power (W)")
    ax2.set_ylabel("Write bandwidth MiB/s)")
    
    ax1.legend(loc=(0,0.87), frameon=False, fontsize=30)
    ax2.legend(loc=(0.52,0.87), frameon=False, fontsize=30)
    plt.title('Random writes')
    plt.show()
    fig.savefig(f'./{OUT_FILE}.pdf',bbox_inches='tight')

set_font_small()
plot_gc('small')
set_font_large()
plot_gc('large')

In [ ]:
# Plot 3 (summarize)
IN_FILES = [f'data/repeated_bs/read_bs_{bs}' for bs in [f"{i}k" for i in range (1,4097)]]
OUT_FILE = "ssd_power_measurement_block_size"

ys_power = [0.] * len(IN_FILES)
for idx,filename in enumerate(IN_FILES):
    _, y = parse_file(filename, X_RANGE, TIME_METRIC)
    ys_power[idx] = np.mean(y)

In [ ]:
# Plot 3 (plot)
def plot_block_size(size_label):
    IN_FILES = [f'data/repeated_bs/read_bs_{bs}' for bs in [f"{i}k" for i in range (1,4097)]]
    OUT_FILE = f"ssd_power_measurement_block_size_{size_label}"
    fig, ax1 = plt.subplots(figsize=(15, 7))
    ax2 = ax1.twinx()
    greys = iter(sns.color_palette("colorblind", num_lines))
    
    ln1 = sns.lineplot(x=range(1,len(IN_FILES)+1), y=ys_power, c=next(greys), label='Power', ax=ax1)
    
    ys = [0.] * len(IN_FILES)
    for jdx,filename in enumerate(IN_FILES):
        try:
            with open(f'{filename}_bw.1.log', 'r') as file:
                data = [line.rstrip() for line in file]
                bw_x = [0.] * len(data)
                bw_y = [0.] * len(data)
                for idx, line in enumerate(data):
                    spl = line.split()
                    bw_x[idx] = (float(spl[0][:-1])) / 1000.
                    bw_y[idx] = (float(spl[1][:-1])) / 1024.
                bw_x = np.array(bw_x)
                bw_y = np.array(bw_y)
                timespan = np.logical_and(bw_x >= X_RANGE[0], bw_x <= X_RANGE[1]) 
                bw_x = bw_x[timespan]
                bw_y = bw_y[timespan]
            
                ys[jdx] = np.mean(bw_y)
        except:
            print(f"missing {filename}")
    
    ln2 = sns.lineplot(x=range(1,len(IN_FILES)+1), y=ys, c=next(greys), label='Bandwidth', ax=ax2)
    
    ax1.set_ylim([0, 7])
    ax1.set_xticks((0, 1024, 2048, 1024*3, 1024*4))
    ax1.set_yticks((0, 1, 2, 3, 4, 5, 6, 7))
    ax2.set_ylim([0, 3400])
    ax2.set_yticks((0, 1000, 2000, 3000))
    ax2.set_yticklabels(['0', '1000', '2000', '3000'])
    ax1.set_xlabel("Request size (KiB)")
    ax1.set_ylabel("Power (W)")
    ax2.set_ylabel("Read bandwidth (MiB/s)")
    
    ax1.legend(loc=(0,0.87), frameon=False, fontsize=30)
    ax2.legend(loc=(0.52,0.87), frameon=False, fontsize=30)
    plt.title('Random reads')
    plt.show()
    fig.savefig(f'./{OUT_FILE}.pdf',bbox_inches='tight')

set_font_small
plot_block_size('small')
set_font_large()
plot_block_size('large')